# Cross-tabulate

Cross-tabulations of 'slash' relationship pairs against the categorical variables.

In [1]:
import pandas as pd
import numpy as np

import ds9

df = ds9.df()

## Explode list `relationshipspairslash` into rows

Dropping the uninteresting columns to keep the size under control

In [2]:
df = df.drop(['cleandate', 'complete', 'language', 'publicationdate', 'summary', 'characters', 'filename', 'parsedate', 'fandoms'], axis=1)

new_df = pd.DataFrame()
for index, relationshipspairslash in zip(df.index, df['relationshipspairslash']):
    for i in relationshipspairslash:
        row = df[df.index == index].copy()
        row['relationshipspairslash'] = i
        new_df = pd.concat([new_df, row], ignore_index=True)
df = new_df

## Cross-tabulate with `rating`

See https://datagy.io/pandas-crosstab/

Note that we can only cross-tabulate categories, so we might need to convert strings to categories.

In [3]:
# We want the category for relationshipspairsslash to be most-common first
# (in stats jargon, descending order of frequency). So we need to compute
# the frequency of relationshipspairsslash before we can define the type
# for the category relationshipspairsslash_type.  As we want that type
# to have an order.
f = df['relationshipspairslash'].value_counts()
f = f.reset_index(name = 'n')
cat_list = f['index'].tolist()
relationshipspairslash_type = pd.api.types.CategoricalDtype(categories=cat_list, ordered=True)
df['relationshipspairslash'] = df['relationshipspairslash'].astype(relationshipspairslash_type)

Pandas has a nice crosstab() function. For categorical data it tabulates the frequency, which is exactly what we want.

In [4]:
crosstab_relationshipspairslash_rating = pd.crosstab(index = df.relationshipspairslash,
                                                     columns = df.rating,
                                                     margins=True,
                                                     margins_name = 'Total')

# Save .CSV
crosstab_relationshipspairslash_rating.to_csv('crosstab-relationshipspairslash-rating.csv')

# Display table
pd.options.display.max_rows = 10000
crosstab_relationshipspairslash_rating

rating,General Audiences,Teen And Up Audiences,Mature,Explicit,Not Rated,Total
relationshipspairslash,,,,,,
Elim Garak/Julian Bashir,1113,1248,592,758,139,3850
Jadzia Dax/Kira Nerys,163,163,37,35,8,406
Odo/Quark,94,144,47,80,7,372
non-cast/non-cast,65,105,60,55,14,299
Kira Nerys/Odo,100,87,23,26,11,247
Julian Bashir/non-cast,31,35,38,36,5,145
Jadzia Dax/Worf,64,46,10,14,10,144
Keiko O'Brien/Miles O'Brien,38,61,10,12,5,126
Corat Damar/Weyoun,13,37,21,51,2,124


In [5]:
crosstab_relationshipspairslash_rating_percent = pd.crosstab(index = df.relationshipspairslash,
                                                             columns = df.rating,
                                                             normalize='index',
                                                             margins=True,
                                                             margins_name = 'Total').round(3)*100
crosstab_relationshipspairslash_rating_percent.to_csv('crosstab-relationshipspairslash-rating-percent.csv')
crosstab_relationshipspairslash_rating_percent

rating,General Audiences,Teen And Up Audiences,Mature,Explicit,Not Rated
relationshipspairslash,,,,,
Elim Garak/Julian Bashir,28.9,32.4,15.4,19.7,3.6
Jadzia Dax/Kira Nerys,40.1,40.1,9.1,8.6,2.0
Odo/Quark,25.3,38.7,12.6,21.5,1.9
non-cast/non-cast,21.7,35.1,20.1,18.4,4.7
Kira Nerys/Odo,40.5,35.2,9.3,10.5,4.5
Julian Bashir/non-cast,21.4,24.1,26.2,24.8,3.4
Jadzia Dax/Worf,44.4,31.9,6.9,9.7,6.9
Keiko O'Brien/Miles O'Brien,30.2,48.4,7.9,9.5,4.0
Corat Damar/Weyoun,10.5,29.8,16.9,41.1,1.6


## Cross-tabulate with `categories`

In [6]:
# Explode categories
# Use `cat_df` as we don't want to damage `df`
cat_df = pd.DataFrame()
for index, categories in zip(df.index, df['categories']):
    for i in categories:
        row = df[df.index == index].copy()
        row['categories'] = i
        cat_df = pd.concat([cat_df, row], ignore_index=True)

# Cross-tabulate
cat_df['categories'] = cat_df['categories'].astype(ds9.categories_type)
crosstab_relationshipspairslash_categories = pd.crosstab(index = cat_df.relationshipspairslash,
                                                         columns = cat_df.categories,
                                                         margins = True,
                                                         margins_name = 'Total')

# Save .CSV
crosstab_relationshipspairslash_categories.to_csv('crosstab-relationshipspairslash-categories.csv')

# Display table
crosstab_relationshipspairslash_categories

categories,M/M,Gen,F/M,F/F,Multi,No category,Other,Total
relationshipspairslash,,,,,,,,
Elim Garak/Julian Bashir,3551,324,200,81,152,104,53,4465
Jadzia Dax/Kira Nerys,46,39,24,369,22,2,5,507
Odo/Quark,338,37,26,11,25,3,14,454
non-cast/non-cast,134,71,143,64,52,8,25,497
Kira Nerys/Odo,42,76,179,19,27,10,31,384
Julian Bashir/non-cast,110,15,40,11,15,5,12,208
Jadzia Dax/Worf,45,40,92,16,22,5,2,222
Keiko O'Brien/Miles O'Brien,59,45,73,20,28,0,3,228
Corat Damar/Weyoun,118,12,6,7,8,0,30,181


In [7]:
# Repeat for percentages
crosstab_relationshipspairslash_categories_percent = pd.crosstab(index = cat_df.relationshipspairslash,
                                                                 columns = cat_df.categories,
                                                                 margins=True,
                                                                 margins_name = 'Total',
                                                                 normalize='index').round(3)*100
crosstab_relationshipspairslash_categories_percent.to_csv('crosstab-relationshipspairslash-categories-percent.csv')
crosstab_relationshipspairslash_categories_percent

categories,M/M,Gen,F/M,F/F,Multi,No category,Other
relationshipspairslash,,,,,,,
Elim Garak/Julian Bashir,79.5,7.3,4.5,1.8,3.4,2.3,1.2
Jadzia Dax/Kira Nerys,9.1,7.7,4.7,72.8,4.3,0.4,1.0
Odo/Quark,74.4,8.1,5.7,2.4,5.5,0.7,3.1
non-cast/non-cast,27.0,14.3,28.8,12.9,10.5,1.6,5.0
Kira Nerys/Odo,10.9,19.8,46.6,4.9,7.0,2.6,8.1
Julian Bashir/non-cast,52.9,7.2,19.2,5.3,7.2,2.4,5.8
Jadzia Dax/Worf,20.3,18.0,41.4,7.2,9.9,2.3,0.9
Keiko O'Brien/Miles O'Brien,25.9,19.7,32.0,8.8,12.3,0.0,1.3
Corat Damar/Weyoun,65.2,6.6,3.3,3.9,4.4,0.0,16.6


## Cross-tabulate with `warnings`

In [8]:
warnings_df = pd.DataFrame()
for index, warnings in zip(df.index, df['warnings']):
    for i in warnings:
        row = df[df.index == index].copy()
        row['warnings'] = i
        warnings_df = pd.concat([warnings_df, row], ignore_index=True)

# Cross-tabulate
warnings_df['warnings'] = warnings_df['warnings'].astype(ds9.warnings_type)
crosstab_relationshipspairslash_warnings = pd.crosstab(index = warnings_df.relationshipspairslash,
                                                         columns = warnings_df.warnings,
                                                         margins = True,
                                                         margins_name = 'Total')

# Save .CSV
crosstab_relationshipspairslash_warnings.to_csv('crosstab-relationshipspairslash-warnings.csv')

# Display table
crosstab_relationshipspairslash_warnings


warnings,No Archive Warnings Apply,Choose Not To Use Archive Warnings,Graphic Depictions Of Violence,Major Character Death,Rape/Non-Con,Underage,Total
relationshipspairslash,,,,,,,
Elim Garak/Julian Bashir,3073,661,74,52,52,6,3918
Jadzia Dax/Kira Nerys,350,50,1,15,1,0,417
Odo/Quark,320,46,3,6,2,0,377
non-cast/non-cast,194,73,23,15,8,3,316
Kira Nerys/Odo,161,77,7,8,3,0,256
Julian Bashir/non-cast,99,29,9,7,9,0,153
Jadzia Dax/Worf,104,34,4,9,1,0,152
Keiko O'Brien/Miles O'Brien,106,16,3,1,1,0,127
Corat Damar/Weyoun,83,27,5,5,8,0,128


In [9]:
crosstab_relationshipspairslash_warnings_percent = pd.crosstab(index = warnings_df.relationshipspairslash,
                                                                 columns = warnings_df.warnings,
                                                                 margins=True,
                                                                 margins_name = 'Total',
                                                                 normalize='index').round(3)*100
crosstab_relationshipspairslash_warnings_percent.to_csv('crosstab-relationshipspairslash-warnings-percent.csv')
crosstab_relationshipspairslash_warnings_percent

warnings,No Archive Warnings Apply,Choose Not To Use Archive Warnings,Graphic Depictions Of Violence,Major Character Death,Rape/Non-Con,Underage
relationshipspairslash,,,,,,
Elim Garak/Julian Bashir,78.4,16.9,1.9,1.3,1.3,0.2
Jadzia Dax/Kira Nerys,83.9,12.0,0.2,3.6,0.2,0.0
Odo/Quark,84.9,12.2,0.8,1.6,0.5,0.0
non-cast/non-cast,61.4,23.1,7.3,4.7,2.5,0.9
Kira Nerys/Odo,62.9,30.1,2.7,3.1,1.2,0.0
Julian Bashir/non-cast,64.7,19.0,5.9,4.6,5.9,0.0
Jadzia Dax/Worf,68.4,22.4,2.6,5.9,0.7,0.0
Keiko O'Brien/Miles O'Brien,83.5,12.6,2.4,0.8,0.8,0.0
Corat Damar/Weyoun,64.8,21.1,3.9,3.9,6.2,0.0
